In [ ]:
# !python -V

### Preprocessing Data

In [ ]:
# import libraries for data preprocessing

import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

from sklearn.metrics import mean_squared_error

In [ ]:
pip install mlflow

In [ ]:
import mlflow

In [ ]:
# mlflow.delete_experiment(0)
#
# mlflow.start_run()
# run = mlflow.active_run()
# print("Active run_id: {}".format(run.info.run_id))
# mlflow.end_run()

In [ ]:
# sqlite is used as mlflow backend and set the experiment name for tracking
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

### Train using Linear Regressor and set the experiment to mlflow manually

In [ ]:
# Preprocessing Data
# function to read the data and remove outliers

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df



In [ ]:
# call read and preprocessing functions for train, validation dataset
df_train = read_dataframe('green_tripdata_2021-01.parquet')
df_val = read_dataframe('green_tripdata_2021-02.parquet')

In [ ]:
# info of dataset
len(df_train), len(df_val)

In [ ]:
# feature engineering
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [ ]:
# fit the dictvectorizer and preprocess data
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
# 1. Train a linear model (without mlflow)
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

# mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
import pickle

In [ ]:
with open('lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
# Train another linear model (Lasso)
# lr = Lasso(0.01)
# lr.fit(X_train, y_train)

# y_pred = lr.predict(X_val)

# mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
# 2. Train a linear model (with mlflow)

with mlflow.start_run():

    mlflow.set_tag("developer", "Amber")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)

    mlflow.log_metric("rmse", rmse)
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")
